In [9]:
import os
from facenet_pytorch import MTCNN
from PIL import Image
import torch
# from imutils.video import FileVideoStream
import cv2
import time
import glob
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [10]:
import logging
import sys
logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [12]:
mtcnn = MTCNN(keep_all=False, select_largest=False, device=device)#post_process=False

In [13]:

def save_faces(interval_id, frame_filename, debug=False):
    frame_id, frame_path = resolve_paths(frame_filename, interval_id)
    
    image = Image.open(frame_path)
    boxes, probs, points = mtcnn.detect(image, landmarks=True)
    img_draw = image.copy()
    draw = ImageDraw.Draw(img_draw)
    
    for i, (box, point) in enumerate(zip(boxes, points)):
        draw.rectangle(box.tolist(), width=5)
        for p in point:
            draw.rectangle((p - 10).tolist() + (p + 10).tolist(), width=10)
        detected_face_path = resolve_detected_face_path(interval_id, frame_id, i, create=True)
        extract_face(image, box, image_size=224, margin=70, save_path=detected_face_path)
    
    annotated_faces_path = resolve_annot_faces_path(interval_id, frame_id)
    img_draw.save(annotated_faces_path)
    
    debug_print(debug, frame_filename, frame_id, annotated_faces_path, probs)

In [15]:
interval_frames_dir = '/Users/staveshemesh/Projects/PATS_DATA/Youtube/oliver/hmoAX9f6MOc/102430/vokens/v6_face_annot_224/'
frame_paths = [os.path.join(interval_frames_dir, filename) for filename in os.listdir(interval_frames_dir)]
frames = [Image.open(frame_path) for frame_path in frame_paths]

In [ ]:
len(frame_paths)

In [16]:
logging.info(f'Start.. {len(frames)} batch')
# Detect face
faces = mtcnn(frames)
faces.shape
logging.info('End.')

2021-05-16 01:07:45,833 | INFO : Start.. 334 batch


KeyboardInterrupt: 

In [ ]:
%time
    
# Detect face
faces = mtcnn(frame)
faces.shape

In [ ]:
%time
    
frames = [Image.open(frame_path) for frame_path in frame_paths]
# Detect face
faces = mtcnn(frame)
faces.shape

In [9]:
class FastMTCNN(object):
    """Fast MTCNN implementation."""
    
    def __init__(self, stride, resize=1, *args, **kwargs):
        """Constructor for FastMTCNN class.
        
        Arguments:
            stride (int): The detection stride. Faces will be detected every `stride` frames
                and remembered for `stride-1` frames.
        
        Keyword arguments:
            resize (float): Fractional frame scaling. [default: {1}]
            *args: Arguments to pass to the MTCNN constructor. See help(MTCNN).
            **kwargs: Keyword arguments to pass to the MTCNN constructor. See help(MTCNN).
        """
        self.stride = stride
        self.resize = resize
        self.mtcnn = MTCNN(*args, **kwargs)
        
    def __call__(self, frames):
        """Detect faces in frames using strided MTCNN."""
        if self.resize != 1:
            frames = [
                cv2.resize(f, (int(f.shape[1] * self.resize), int(f.shape[0] * self.resize)))
                    for f in frames
            ]
                      
        boxes, probs = self.mtcnn.detect(frames[::self.stride])

        faces = []
        for i, frame in enumerate(frames):
            box_ind = int(i / self.stride)
            if boxes[box_ind] is None:
                continue
            for box in boxes[box_ind]:
                box = [int(b) for b in box]
                faces.append(frame[box[1]:box[3], box[0]:box[2]])
        
        return faces

In [11]:
fast_mtcnn = FastMTCNN(
    stride=4,
    resize=224,
    margin=70,
    factor=0.6,
    keep_all=True,
    device=device
)

In [13]:
def run_detection(fast_mtcnn, filenames):
    frames = []
    frames_processed = 0
    faces_detected = 0
    batch_size = 60
    start = time.time()

    for filename in tqdm(filenames):

        v_cap = FileVideoStream(filename).start()
        v_len = int(v_cap.stream.get(cv2.CAP_PROP_FRAME_COUNT))

        for j in range(v_len):

            frame = v_cap.read()
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)

            if len(frames) >= batch_size or j == v_len - 1:

                faces = fast_mtcnn(frames)

                frames_processed += len(frames)
                faces_detected += len(faces)
                frames = []

                print(
                    f'Frames per second: {frames_processed / (time.time() - start):.3f},',
                    f'faces detected: {faces_detected}\r',
                    end=''
                )

        v_cap.stop()

In [14]:
run_detection(fast_mtcnn, filenames)


NameError: name 'filenames' is not defined